# Fast Fourier Transform & Denoising


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from numpy.fft import *
import pyarrow.parquet as pq
import matplotlib.pyplot as plt

sns.set_style("whitegrid")

## 1 - Loading Data

### Signals

In [ ]:
signals = pq.read_table('../input/train.parquet', columns=[str(i) for i in range(999)]).to_pandas()

In [ ]:
signals = np.array(signals).T.reshape((999//3, 3, 800000))

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(signals[0, 0, :], label='Phase 0')
plt.plot(signals[0, 1, :], label='Phase 1')
plt.plot(signals[0, 2, :], label='Phase 2')
plt.legend()
plt.show()

### Target

In [ ]:
train_df = pd.read_csv('../input/metadata_train.csv')
train_df.head()

In [ ]:
target = train_df['target'][::3]

In [ ]:
plt.figure(figsize=(15, 10))
sns.countplot(target)
plt.show()

## 2 - Smoothing by mean
K neighbour ile signal gürültüsünü azaltmak hedeflenmiştir

In [ ]:
def sample(signal, kernel_size):
    sampled = np.zeros((signal.shape[0], signal.shape[1], signal.shape[2]//kernel_size))
    for i in range(signal.shape[2]//kernel_size):
        begin = kernel_size * i
        end = min(kernel_size * (i + 1), signal.shape[2])
        sampled[:, :, i] = np.mean(signal[:, :, begin:end], axis=2)
    return sampled

In [ ]:
sampled = sample(signals, 100)

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(sampled[0, 0, :], label='Phase 0')
plt.plot(sampled[0, 1, :], label='Phase 1')
plt.plot(sampled[0, 2, :], label='Phase 2')
plt.legend()
plt.show()

In [ ]:
def filter_signal(signal, threshold=1e8):
    fourier = rfft(signal)
    frequencies = rfftfreq(signal.size, d=20e-3/signal.size)
    fourier[frequencies > threshold] = 0
    return irfft(fourier)

### Testing some thresholds

In [ ]:
filtered = filter_signal(signals[0, 0, :], threshold=1e3)

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(signals[0, 0, :], label='Raw')
plt.plot(filtered, label='Filtered')
plt.legend()
plt.title("FFT Denoising with threshold = 1e3", size=15)
plt.show()

In [ ]:
filtered = filter_signal(signals[0, 0, :], threshold=1e5)

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(signals[0, 0, :], label='Raw')
plt.plot(filtered, label='Filtered')
plt.legend()
plt.title("FFT Denoising with threshold = 1e5", size=15)
plt.show()

In [ ]:
filtered = filter_signal(signals[0, 0, :], threshold=1e7)

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(signals[0, 0, :], label='Raw')
plt.plot(filtered, label='Filtered')
plt.legend()
plt.title("FFT Denoising with threshold = 1e7", size=15)
plt.show()